Collision

Notebook for collision modeling in ASPECT 2026

In [1]:
import os, sys, shutil, math
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import datetime
import subprocess

# Include this pakage
HaMaGeoLib_DIR = "/home/lochy/ASPECT_PROJECT/HaMaGeoLib"
if os.path.abspath(HaMaGeoLib_DIR) not in sys.path:
    sys.path.append(os.path.abspath(HaMaGeoLib_DIR))
from hamageolib.utils.exception_handler import my_assert

# Working directories
local_Collision_dir = "/mnt/lochy/ASPECT_DATA/Collision0" # data directory
# remote_ThDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/ThDSubduction"
assert(os.path.isdir(local_Collision_dir))

# py_temp file and temperature results directory
py_temp_dir = os.path.join(HaMaGeoLib_DIR, "py_temp_files")
RESULT_DIR = os.path.join(HaMaGeoLib_DIR, 'results')
os.makedirs(py_temp_dir, exist_ok=True) # Ensure the directory exists

# paraview scripts directory
SCRIPT_DIR = os.path.join(HaMaGeoLib_DIR, "scripts")

today_date = datetime.datetime.today().strftime("%Y-%m-%d") # Get today's date in YYYY-MM-DD format
py_temp_file = os.path.join(py_temp_dir, f"py_temp_{today_date}.sh")

if not os.path.exists(py_temp_file):
    bash_header = """#!/bin/bash
# =====================================================
# Script: py_temp.sh
# Generated on: {date}
# Description: Temporary Bash script created by Python
# =====================================================

""".format(date=today_date)
    with open(py_temp_file, "w") as f:
        f.write(bash_header)

print(f"File ensured at: {py_temp_file}")

File ensured at: /home/lochy/ASPECT_PROJECT/HaMaGeoLib/py_temp_files/py_temp_2025-10-30.sh


# Cases

We now begin processing the simulation cases.
The first step is to enumerate the case directories and confirm that all specified paths exist on the filesystem.

Options:

* do_post_process - control the run of the following code blocks


In [2]:
do_post_process = True

In [3]:
if do_post_process:
    # placeholder for a 3-D case
    case_name = None 
    # Initial test from Fritz
    case_name_2d = "collision_test/Fritz_test0"; prm_basename_2d = "init_compressible_test.prm"; wb_basename_2d = "original.wb"; output_directory="isentropic_adiabat_new_boundary"

    local_dir = None; local_dir_2d = None
    if case_name is not None:
        local_dir = os.path.join(local_Collision_dir, case_name)
        assert(os.path.isdir(local_dir))
        print("local_dir:\n\t", local_dir)
        subprocess.run(['mkdir', '-p', '%s/img/pv_outputs' % local_dir])
    if case_name_2d is not None:
        local_dir_2d = os.path.join(local_Collision_dir, case_name_2d)
        assert(os.path.isdir(local_dir_2d))
        print("local_dir_2d:\n\t", local_dir_2d)
        subprocess.run(['mkdir', '-p', '%s/img/pv_outputs' % local_dir_2d])

local_dir_2d:
	 /mnt/lochy/ASPECT_DATA/Collision0/collision_test/Fritz_test0


## Generate Paraview script

We apply a combined workflow with **ParaView**.  
In this notebook, we generate the processing script.  
We then use ParaView to refine styling and export the final plots.

options

- `is_prepare_for_plot` — Generate the **ParaView** Python script (no data pre-processing).  
- `is_process_pyvista_for_plot` — Pre-process results with **PyVista** before exporting the ParaView script, then proceed in ParaView.


In [4]:
is_prepare_for_plot = True
is_process_pyvista_for_plot = False

In [5]:
if do_post_process and is_prepare_for_plot:

    from hamageolib.research.haoyuan_collision0.case_options import CASE_OPTIONS_TWOD
    from hamageolib.research.haoyuan_collision0.post_process import ProcessVtuFileTwoDStep, GenerateParaviewScript

    # check again case directory exists    
    assert(local_dir_2d is not None)

    # options 
    graphical_step = 0

    # original script
    ofile_list = ["collision0.py"]; require_base=True

    # automatically read case configurations
    Case_Options_2d = CASE_OPTIONS_TWOD(local_dir_2d, case_file=prm_basename_2d, wb_basename=wb_basename_2d, output_directory=output_directory)
    Case_Options_2d.Interpret()
    Case_Options_2d.SummaryCaseVtuStep(os.path.join(local_dir_2d, "summary.csv"))

    # the index of the vtu file depends on whether all the intial adaptive steps are output
    pvtu_step = Case_Options_2d.get_pvtu_step(graphical_step)

    ProcessVtuFileTwoDStep(local_dir_2d, pvtu_step, Case_Options_2d)

    GenerateParaviewScript(local_dir_2d, Case_Options_2d, ofile_list, require_base=require_base)

	 File generated: /mnt/lochy/ASPECT_DATA/Collision0/collision_test/Fritz_test0/paraview_scripts/collision0.py
